In [ ]:
! pip install selenium
! pip install pandas

In [66]:
bus_routes =[]
bus_details =[]
# empty list for storing the collected routes

In [67]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://www.redbus.in/')

States = ['APSRTC',
          'KSRTC (Kerala)',
          'TSRTC',
          'Kadamba Transport Corporation Limited (KTCL)',
          'RSRTC',
          'South Bengal State Transport Corporation (SBSTC)',
          'HRTC',
          'Assam State Transport Corporation (ASTC)',
          'UPSRTC',
          'WBTC (CTC)',
          'PEPSU (Punjab)']
# select the states to scrap the routes

def collect_bus_routes_links(current_state):
            
    bus_route = driver.find_elements(By.XPATH,"//a[@class ='route']")
    route_links = driver.find_elements(By.XPATH,"//div[@class = 'route_details']")
    
    for i in range(len(bus_route)):
        bus_RT = bus_route[i].text

        links = route_links[i].find_elements(By.TAG_NAME,"a")
        for link in links:
            href = link.get_attribute("href")
            
            bus_rts = {
                "State": current_state,
                "Routes": bus_RT,
                "Route_links": href
            }
            bus_routes.append(bus_rts)

# wrote this function to reuse code and
# collecting the bus routes data from the page and appending only the text to the empty list

def page_navigation():
    page_number = 2
    # created and assigned the variable to loop the page number

    while True:

        try:
            path = f"//div[@class='DC_117_pageTabs ' and contains(text(), '{page_number}')]"
            # path for the pages

            driver.find_element(By.XPATH,path).click()
            # navigating to the secone page

            collect_bus_routes_links(state)
            # function called and pages data collected

            page_number = page_number + 1
            # increment to navigate the second page
            # loop continue until the end page
            
        except Exception:
            break
        # if there is no other pages in routes this except will break the condition

for state in States:

    driver.get("https://www.redbus.in/online-booking/rtc-directory")
    driver.execute_script("document.body.style.zoom='0.6';")
    # pass the link where we need to scrap the data
    time.sleep(1)
    # wait 1 sec to load the web page

    #driver.execute_script('window.scrollBy(0, 210)')
    time.sleep(1)

    path = f"{state}"
    routes = driver.find_element(By.LINK_TEXT, path).click()
    driver.execute_script("document.body.style.zoom='0.8';")
    time.sleep(0.5)
    # finding the state and click to collect the data
    
    collect_bus_routes_links(state)
    # collecting the data in single page
    
    driver.execute_script('window.scrollBy(0, 1500)')
    time.sleep(1)

    page_navigation()

        


In [68]:

def scrolling_the_page():

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Scroll down to the bottom
        
        time.sleep(0.5)
        # Wait for new elements to load
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        # Calculate new scroll height and compare with the last scroll height
        
        if new_height == last_height:  # Break if no more buses are loaded
            break
        last_height = new_height






def bus_details_scraping(current_route,current_state,current_link):
    time.sleep(1)
    all_buses = driver.find_element(By.XPATH, "//div[@id='result-section']")
    li_elements = all_buses.find_elements(By.CSS_SELECTOR, "li.row-sec")
    # After scrolling, find all <li> elements within the <ul>

    for li in li_elements:
        bus_name = li.find_element(By.CSS_SELECTOR, ".travels").text # bus name
        bus_types = li.find_element(By.CSS_SELECTOR, ".bus-type").text # bus type
        dp_time = li.find_element(By.CSS_SELECTOR, ".dp-time").text # depart time
        dur = li.find_element(By.CSS_SELECTOR, ".dur").text # duration
        re_time = li.find_element(By.CSS_SELECTOR, ".bp-time").text # reach time 
        seat = li.find_element(By.CSS_SELECTOR, ".seat-left").text # seat available
        ratings = li.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']//span") # star rating

        if ratings:
            for rating in ratings:
                star_rating = rating.text  # Get the star rating
        else:
            star_rating = 0.0
            
        fare = li.find_element(By.CSS_SELECTOR, ".fare span.f-19").text # price

        
        bus_items = ({
            "States": current_state,
            "Route_link": current_link,
            "Routes_Name": current_route,
            "Bus_Name" : bus_name,
            "Bus_Type": bus_types,
            "Departing_Time": dp_time,
            "Duration": dur,
            "Reaching_Time": re_time,
            "Price":fare,
            "Seat_Availability": seat,
            "Star_Rating": star_rating,
        })
        bus_details.append(bus_items)


In [ ]:
bus_routes
# to check how many routes are there

In [71]:
# all states bus details scraping

driver = webdriver.Chrome()
driver.maximize_window()

for item in bus_routes:
    link = item['Route_links']
    route = item['Routes']
    states =item['State']
    # looping the links for route to collect the bus details

    try:
        try:
            driver.get(link)
            time.sleep(5)

            try:
                driver.find_element(By.XPATH, "(//div[@class = 'button' and contains(text(), 'View Buses')])[2]").click()
                time.sleep(1.5)
                driver.find_element(By.XPATH, "//div[@class = 'button' and contains(text(), 'View Buses')]").click()

            except:
                driver.find_element(By.XPATH, "//div[@class = 'button' and contains(text(), 'View Buses')]").click()
                time.sleep(1)
                
            scrolling_the_page()
            bus_details_scraping(route,states,link)

        except Exception:
            scrolling_the_page()
            bus_details_scraping(route,states,link)
            
    except Exception:
        pass

driver.quit()


In [ ]:
len(bus_details)
# to check how many buses details are scraped

In [74]:
import pandas as pd

redbus_data = pd.DataFrame(bus_details)
# changing the list of dict to dataframe

len(redbus_data)

redbus_data.to_csv('Redbus_data.csv')
# convert the dataframe to csv for local storage


In [ ]:
redbus_data
# view dataframe

In [76]:
import mysql.connector as db

db_connection = db.connect(
    host = "localhost",
    user = "root",
    password = "iamveryhappy123@",
    database = "redbus_data"
)
# connect the MySQL localhost databases

In [77]:
curr = db_connection.cursor()
# make connection

In [82]:
create_table_query = """
CREATE TABLE redbus_data.bus_routes(
             Id INT AUTO_INCREMENT primary key,
             States varchar(100),
             Route_link varchar(100),
             Routes_Name varchar(100),
             Bus_Name varchar(100),
             Bus_Type varchar(100),
             Departing_Time time,
             Duration varchar(100),
             Reaching_Time time,
             Price decimal(10,3),
             Seat_Availability varchar(100),
             Star_Rating float(3,2)
             );
"""
curr.execute(create_table_query)

# create empty table and add columns that contain previous DataFrame

In [ ]:
import pandas as pd

# Replace empty strings or NaN values in 'Price' and 'Star_Rating' columns with default values or handle as needed
redbus_data['Price'] = pd.to_numeric(redbus_data['Price'], errors='coerce').fillna(0.0)
redbus_data['Star_Rating'] = pd.to_numeric(redbus_data['Star_Rating'], errors='coerce').fillna(0.0)

# Check the rest of the columns to ensure there are no empty or NaN values where not allowed
redbus_data.fillna('', inplace=True)  # Replace remaining NaNs with empty strings for non-numeric fields

# Prepare the SQL insert query
insert_query = """
INSERT INTO redbus_data.bus_routes
    (States, Route_link, Routes_Name, Bus_Name, Bus_Type, Departing_Time, Duration, Reaching_Time, Price, Seat_Availability, Star_Rating)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Loop through the DataFrame and insert rows into the database
for _, row in redbus_data.iterrows():
    curr.execute(insert_query, tuple(row))

# Commit the transaction to save changes in the database
db_connection.commit()

print("Data inserted successfully!")


In [ ]:
import pandas as pd

# Replace empty strings or NaN values in 'Price' and 'Star_Rating' columns with default values or handle as needed
redbus_data['Price'] = pd.to_numeric(redbus_data['Price'], errors='coerce').fillna(0.0)
redbus_data['Star_Rating'] = pd.to_numeric(redbus_data['Star_Rating'], errors='coerce').fillna(0.0)

# Check the rest of the columns to ensure there are no empty or NaN values where not allowed
redbus_data.fillna('', inplace=True)  # Replace remaining NaNs with empty strings for non-numeric fields

# Prepare the SQL insert query
insert_query = """
INSERT INTO redbus_data.bus_routes
    (States, Route_link, Routes_Name, Bus_Name, Bus_Type, Departing_Time, Duration, Reaching_Time, Price, Seat_Availability, Star_Rating)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Loop through the DataFrame and insert rows into the database
for _, row in redbus_data.iterrows():
    curr.execute(insert_query, tuple(row))

# Commit the transaction to save changes in the database
db_connection.commit()

print("Data inserted successfully!")


In [85]:
curr.close()
db_connection.close()
# finally close the all databases connections